# Assignment: Automate Review Classification with LangGraph


---

### Objective:
This assignment challenges you to **design and implement a robust review classification workflow using LangGraph**. You'll leverage LangGraph's state management and conditional routing capabilities to create an automated agent that can classify customer reviews (e.g., positive, negative, neutral) and handle ambiguous cases by requesting more information or escalating. This project will solidify your understanding of building sophisticated, multi-step LLM applications.

---

### Instructions:
1.  **LLM Access**: You'll need access to an LLM API (e.g., Google's Gemini, OpenAI's GPT-4). For this assignment, we'll primarily use **Google's Gemini Pro model** via the `langchain-google-genai` integration.
2.  **Environment Setup**: Install the necessary Python libraries: `pip install langchain-google-genai langchain_community langgraph pydantic`.
3.  **API Key**: Securely handle your API key. It's best practice to load it from an environment variable.
4.  **Jupyter Notebook**: All your code, outputs, workflow diagrams (if you generate them), observations, and analysis must be documented in this Jupyter Notebook.
5.  **Review Samples**: Prepare a set of diverse customer reviews for testing (positive, negative, neutral, and ambiguous ones).
6.  **Analysis**: Critically evaluate your LangGraph workflow's ability to classify reviews and handle different scenarios.

---

## Part 1: Setup and Workflow State Definition
Begin by configuring your LLM and defining the state that your LangGraph workflow will manage.

### Task 1.1: API Configuration and LLM Initialization
Set up your Google Generative AI API key and initialize the `ChatGoogleGenerativeAI` model.

In [ ]:
import os
from typing import Literal
from typing_extensions import TypedDict

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

from langgraph.graph import StateGraph, END

# --- YOUR API KEY HERE ---
# It's highly recommended to load your API key from an environment variable for security.
# For example:
# os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
# For this assignment, you can temporarily paste it directly, but be careful not to share your notebook with the key.
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY_HERE" # Replace with your actual API key

# Initialize the LLM (Gemini Pro)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.0) # Low temperature for classification tasks

print("LLM initialized with Gemini Pro!")

### Task 1.2: Define Workflow State
Define a `TypedDict` to represent the state of your LangGraph workflow. This state will hold information as the review progresses through the classification process.

* **Requirements for State**:
    * `review_text`: The original customer review string.
    * `classification`: The predicted sentiment (e.g., "Positive", "Negative", "Neutral", "Ambiguous"). This should be optional as it's determined later.
    * `clarification_needed`: A boolean indicating if more information is required for ambiguous reviews.
    * `escalation_reason`: An optional string describing why a review was escalated (e.g., "Ambiguous sentiment", "Requires human intervention").

In [ ]:
class ReviewClassificationState(TypedDict):
    review_text: str
    classification: Literal["Positive", "Negative", "Neutral", "Ambiguous", "Escalated", "N/A"] # N/A for initial state
    clarification_needed: bool
    escalation_reason: str # Default to empty string

print("ReviewClassificationState defined!")

---

## Part 2: Define Nodes for the Workflow
Create the individual nodes (functions) that represent steps in your classification process.

### Task 2.1: `classify_review` Node
This node will use the LLM to classify the sentiment of a review. It should output one of "Positive", "Negative", "Neutral", or "Ambiguous".

* **Prompt Design**: Create a prompt that clearly instructs the LLM to classify the review and specify the allowed output categories. Consider using Pydantic for structured output.
* **Output**: The node should update the `classification` field in the state.

In [ ]:
class ClassificationOutput(BaseModel):
    sentiment: Literal["Positive", "Negative", "Neutral", "Ambiguous"]
    reason: str = Field(description="Brief reason for the sentiment classification.")

classification_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert review sentiment classifier. Classify the user's review into one of 'Positive', 'Negative', 'Neutral', or 'Ambiguous'. Provide a brief reason."),
    ("user", "Review: {review_text}")
]).partial(
    format_instructions=ClassificationOutput.schema_json()
)

classification_chain = classification_prompt | llm.with_structured_output(ClassificationOutput)

def classify_review_node(state: ReviewClassificationState) -> ReviewClassificationState:
    print("\n--- Node: Classifying Review ---")
    review_text = state["review_text"]

    try:
        classification_result = classification_chain.invoke({"review_text": review_text})
        print(f"Raw classification result: {classification_result}")

        # Update the state based on the classification result
        new_classification = classification_result.sentiment

        return {
            "classification": new_classification,
            "clarification_needed": (new_classification == "Ambiguous"),
            "escalation_reason": "" # Clear any previous escalation reason
        }
    except Exception as e:
        print(f"Error classifying review: {e}")
        # Fallback to an error state or escalation if classification fails
        return {
            "classification": "Escalated",
            "clarification_needed": False,
            "escalation_reason": f"Classification failed: {e}"
        }

print("classify_review_node defined!")

### Task 2.2: `request_clarification` Node
This node is called if the review is classified as "Ambiguous". It should generate a follow-up question to ask the customer or an internal note for a human agent.

* **Prompt Design**: Craft a prompt that takes the ambiguous review and generates a concise question to clarify its sentiment.
* **Output**: The node should update the `escalation_reason` field with the clarification question, and set `clarification_needed` to `True` (though it should already be true). This is where a real system might send an email or trigger an external action.

In [ ]:
clarification_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. The following review is ambiguous. Generate a concise question to clarify its sentiment or intent. The question should be directed to the user, or an internal note for a human agent."),
    ("user", "Ambiguous Review: {review_text}")
])

clarification_chain = clarification_prompt | llm

def request_clarification_node(state: ReviewClassificationState) -> ReviewClassificationState:
    print("\n--- Node: Requesting Clarification ---")
    review_text = state["review_text"]

    try:
        clarification_question = clarification_chain.invoke({"review_text": review_text}).content
        print(f"Clarification requested: {clarification_question}")
        return {
            "classification": "Ambiguous",
            "clarification_needed": True,
            "escalation_reason": f"Clarification needed: {clarification_question}"
        }
    except Exception as e:
        print(f"Error requesting clarification: {e}")
        return {
            "classification": "Escalated",
            "clarification_needed": False,
            "escalation_reason": f"Failed to request clarification: {e}"
        }

print("request_clarification_node defined!")

### Task 2.3: `escalate_review` Node
This node handles reviews that cannot be automatically classified or require human attention. It should set the `escalation_reason`.

* **Purpose**: This node simply marks the review for human review.
* **Output**: It should update the `classification` to "Escalated" and provide a suitable `escalation_reason`.

In [ ]:
def escalate_review_node(state: ReviewClassificationState) -> ReviewClassificationState:
    print("\n--- Node: Escalating Review ---")
    # This node is triggered when classification is needed but ambiguous, or direct escalation
    current_classification = state.get("classification", "N/A")
    current_reason = state.get("escalation_reason", "")

    if current_classification == "Ambiguous":
        reason = current_reason if current_reason else "Sentiment is ambiguous and requires human review."
    else:
        reason = current_reason if current_reason else "Review requires manual review due to unexpected scenario or LLM failure."

    print(f"Review escalated. Reason: {reason}")
    return {
        "classification": "Escalated",
        "clarification_needed": False, # Once escalated, clarification isn't needed by the system, but by human
        "escalation_reason": reason
    }

print("escalate_review_node defined!")

---

## Part 3: Constructing the LangGraph Workflow
Combine your nodes into a directed graph using LangGraph's `StateGraph`.

### Task 3.1: Define Edges and Conditional Logic
Define the graph's nodes, entry point, and the conditional edges that determine the flow based on the review classification.

* **Graph Flow**:
    1.  Start at `classify_review_node`.
    2.  From `classify_review_node`:
        * If classification is "Positive", "Negative", or "Neutral", end the workflow.
        * If classification is "Ambiguous", go to `request_clarification_node`.
        * If classification is "Escalated" (e.g., due to an error in `classify_review_node`), go to `escalate_review_node`.
    3.  From `request_clarification_node`: Go to `escalate_review_node` (as automated clarification isn't handled in this basic example, it always escalates for human review after generating the clarification question).
    4.  From `escalate_review_node`: End the workflow.

In [ ]:
# Define the graph
workflow = StateGraph(ReviewClassificationState)

# Add nodes
workflow.add_node("classify", classify_review_node)
workflow.add_node("clarify", request_clarification_node)
workflow.add_node("escalate", escalate_review_node)

# Set entry point
workflow.set_entry_point("classify")

# Define conditional edge from classify
def route_classification(state: ReviewClassificationState) -> Literal["clarify", "escalate", END]:
    classification = state["classification"]
    if classification == "Ambiguous":
        return "clarify"
    elif classification == "Escalated":
        return "escalate"
    else:
        return END # Positive, Negative, Neutral reviews end here

workflow.add_conditional_edges(
    "classify",
    route_classification,
    {
        "clarify": "clarify",
        "escalate": "escalate",
        END: END,
    },
)

# Define edges from clarify and escalate
workflow.add_edge("clarify", "escalate") # After requesting clarification, it's considered escalated for human
workflow.add_edge("escalate", END)

# Compile the graph
app = workflow.compile()

print("LangGraph workflow compiled!")

# Optional: Visualize the graph (requires 'graphviz' and 'pydotplus')
# try:
#     from IPython.display import Image, display
#     display(Image(app.get_graph().draw_png()))
# except ImportError:
#     print("Install graphviz and pydotplus for visualization: pip install graphviz pydotplus")

---

## Part 4: Testing the LangGraph Workflow
Test your workflow with various review samples to ensure it behaves as expected.

### Task 4.1: Test Cases
Run the workflow with the following types of reviews. For each, print the final state of the workflow and any intermediate steps you observe.

* **Test Reviews**:
    1.  **Positive**: "This product is absolutely amazing! Exceeded all my expectations. Five stars!"
    2.  **Negative**: "Worst customer service ever. My order was late and damaged. Highly disappointed."
    3.  **Neutral**: "The delivery was on time, and the packaging was adequate. The item itself is functional."
    4.  **Ambiguous**: "It's okay, I guess. Not great, but not terrible. I'm just very confused about how to use it."
    5.  **Very Short/Unclear**: "Meh."
    6.  **Edge Case/Complex**: "I love the design, but it broke after one week and the return process is a nightmare. I want a refund but also think it's a great concept."

In [ ]:
def run_workflow_and_print_state(review_text: str):
    print(f"\n\n=========== Processing Review: '{review_text}' ===========")
    initial_state = {
        "review_text": review_text,
        "classification": "N/A",
        "clarification_needed": False,
        "escalation_reason": ""
    }

    try:
        for state in app.stream(initial_state):
            print(f"Current state: {state}")
        print(f"Final state for '{review_text}': {state}")
    except Exception as e:
        print(f"An error occurred during workflow execution: {e}")

# --- Run your test cases ---
test_reviews = [
    "This product is absolutely amazing! Exceeded all my expectations. Five stars!", # Positive
    "Worst customer service ever. My order was late and damaged. Highly disappointed.", # Negative
    "The delivery was on time, and the packaging was adequate. The item itself is functional.", # Neutral
    "It's okay, I guess. Not great, but not terrible. I'm just very confused about how to use it.", # Ambiguous
    "Meh.", # Very Short/Unclear
    "I love the design, but it broke after one week and the return process is a nightmare. I want a refund but also think it's a great concept." # Edge Case/Complex
]

for review in test_reviews:
    run_workflow_and_print_state(review)

### Analysis for Task 4.1:
* For each test review, did the workflow produce the expected classification (Positive, Negative, Neutral, Ambiguous, Escalated)?
* Did the ambiguous and edge cases correctly trigger the `clarify` and `escalate` nodes? What clarification questions or escalation reasons were generated?
* Were there any unexpected behaviors or misclassifications? If so, why do you think they occurred?
* Discuss the advantages of using LangGraph (or similar state-machine libraries) for complex LLM workflows compared to simple sequential chains.

---

## Part 5: Conclusion and Reflection
In this markdown cell, provide a comprehensive summary of your findings and reflections based on this assignment.

* **Key Learnings about LangGraph**: What were your main takeaways regarding LangGraph's utility for building multi-step LLM applications?
* **Challenges Encountered**: What difficulties did you face during the assignment (e.g., defining state, routing logic, debugging)?
* **Workflow Improvements**: If you were to enhance this review classification workflow, what specific new nodes, states, or conditional logic would you add (e.g., human-in-the-loop for ambiguous reviews, specific product category classification, integrating external tools)?
* **Practical Applications**: How can automated review classification be applied in real-world business scenarios (e.g., customer support routing, product analytics, marketing insights)?
* **Ethical Considerations**: Discuss any ethical implications related to automating sentiment analysis or classification, such as potential for bias in the LLM, misinterpretation of nuanced language, or impact on customer relations.

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* Save your Jupyter Notebook as `[YourName]_LangGraph_Review_Classification_Assignment.ipynb`.